In [39]:
import sys
print(sys.executable)

!{sys.executable} -m pip list

c:\Users\MANEL\anaconda3\python.exe
Package                           Version
--------------------------------- -------------------
aext-assistant                    4.20.0
aext-assistant-server             4.20.0
aext-core                         4.20.0
aext-core-server                  4.20.0
aext_environments_server          4.20.0
aext-panels                       4.20.0
aext-panels-server                4.20.0
aext-project-filebrowser-server   4.20.0
aext-share-notebook               4.20.0
aext-share-notebook-server        4.20.0
aext-shared                       4.20.0
aext-toolbox                      4.20.0
aiobotocore                       2.19.0
aiohappyeyeballs                  2.4.4
aiohttp                           3.11.10
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.16.5
altair                            5.5.0
anaconda-anon-usage               0.7.1
anaconda-au

In [40]:
import requests
import bs4
import re
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import math
from typing import Callable, Any


In [41]:
# on sépare les biens en catégorie (appartement, maison, terrain, etc.)
# on fait une liste afin de les reconnaitre
# sur la page internet, la recherche est différenciée par des sous ensembles
# th#list : appartement
# tf#list : maison
# tl#list : terrain
# tc#list : commerce
list_bien = ["th", "tf", "tl", "tc"]
# nombres disponibles sur le sites
#appartement : 243 024
#maison : 363 730
# terrain : 91 738
# commerce : 41 587

In [42]:
# Regex pré-compilées (pour éviter les appels et créations incessants)
RE_PRICE   = re.compile(r"(\d[\d\s\u00A0]*)")
RE_SURFACE = re.compile(r"(\d[\d\s\u00A0]*)")
RE_GARDEN  = re.compile(r"\d[\d\u00A0]*")
RE_ROOM    = re.compile(r"\d+")
RE_LOC     = re.compile(r"—\s*(.*?)\s+(\d{5})\s*—")

In [43]:
# on crée une liste des départements français
lst_dep = [f"{i:02d}" for i in range(1, 96)]
lst_dep.remove("20")  # on enlève la corse pour l'instant

In [44]:
# différents prix
list_prix_min = ["50000", "75000", "100000", "120000", "140000", "160000", "180000", "200000", "240000", "260000", "280000", "300000", "325000", "350000", "375000", "400000", "450000", "500000", "600000", "700000", "800000", "900000", "1000000"]
list_prix_max = ["-75000", "-100000", "-120000", "-140000", "-160000", "-180000", "-200000", "-240000", "-260000", "-280000", "-300000", "-325000", "-350000", "-375000", "-400000", "-450000", "-500000", "-600000", "-700000", "-800000", "-900000", "-1000000", ""]

In [45]:
# fonction de récupération de la page html
def get_page(url_request):
    request_text = requests.get(
        url_request,
        headers={"User-Agent": "Python for data science 'appart project'"}
    ).content
    page = bs4.BeautifulSoup(request_text, "lxml")
    return page

In [46]:
def scrap_pages(max_pages: int, url: str) -> list[str]:
    hrefs = []
    for _ in range(max_pages):
        main_page = get_page(url)

        ep_search_list_wrapper = main_page.find("div", {"class": "ep-search-list-wrapper"})
        if ep_search_list_wrapper is None:
            break  

        for a in ep_search_list_wrapper.find_all("a", href=True):
            hrefs.append(a["href"])

        class_next_page = main_page.find("div", {"class": "ep-nav-next"})
        if not class_next_page:
            break

        a_next_page = class_next_page.find("a", href=True)
        if not a_next_page:
            break

        url = a_next_page["href"]

    return hrefs



In [47]:
def scrape_biens(dep: str, bien_code: str, prix_min: str, prix_max: str) -> list[str]:
    date_order = ['.odd.g1', '.oda.g1']
    max_page = 30

    url1 = f"https://www.etreproprio.com/annonces/{bien_code}.p{prix_min}{prix_max}.ld{dep}{date_order[0]}#list"
    main_page = get_page(url1)

    # nombre d'annonces
    nbr_annonces = 0
    ep_count = main_page.find("h1", {"class": "ep-count title-underline"})
    if ep_count is not None:
        txt = ep_count.get_text(" ", strip=True)
        match = re.search(r"(\d+)\s+annonces", txt)
        nbr_annonces = int(match.group(1)) if match else 0

    hrefs = scrap_pages(max_page, url1)

    if nbr_annonces > 600:
        print(nbr_annonces)
        nbr_annonce_rest = nbr_annonces - 600
        nbr_page_rest = math.ceil((nbr_annonce_rest ) / 20)
        print(f"On cherche les annonces par date décroissante sur {nbr_page_rest} pages ({nbr_annonce_rest} annonces)")
        url2 = f"https://www.etreproprio.com/annonces/{bien_code}.p{prix_min}{prix_max}.ld{dep}{date_order[1]}#list"
        hrefs.extend(scrap_pages(nbr_page_rest, url2))
        
    return hrefs
    

In [48]:
def infer_type_from_href(href: str) -> str | None:
    href_l = href.lower()

    # 1) Terrain
    if "terrain" in href_l:
        return "terrain"

    # 2) Maison
    if "maison" in href_l:
        return "maison"

    # 3) Appartement
    if "appartement" in href_l:
        return "appartement"

    # 4) Commerce / local pro (tc)
    commerce_keywords = [
        "commerce", "commercial", "commerciaux",
        "restauration", "restaurant",
        "bureau", "bureau-local",
        "local", "cave", "boutique"
    ]
    if any(k in href_l for k in commerce_keywords):
        return "commerce"

    return None

In [49]:

def extract_fn(href: str):
    """Retourne un dict avec les infos de l'annonce, ou None si invalide."""

    type_bien = infer_type_from_href(href)
    if type_bien is None:
        return None  # ou continue

    page = get_page(href)
    if page is None:
        return None

    ep_price = page.find("div", {"class": "ep-price"})
    if ep_price is None:
        return None

    m_price = RE_PRICE.search(ep_price.get_text(" ", strip=True))
    if not m_price:
        return None

    # éléments principaux
    ep_dtl = page.find("div", {"class": "ep-area"})
    ep_room = page.find("div", {"class": "ep-room"})
    ep_loc = page.find("div", {"class": "ep-loc"})

    if ep_dtl is None or ep_loc is None:
        # pas assez d'infos, on skip
        return None

    # prix (string numérique sans espaces insécables)
    price = m_price.group(1).replace(" ", "").replace("\u00A0", "")

    # surface principale
    m_surface = RE_SURFACE.search(ep_dtl.get_text(" ", strip=True))
    surface = m_surface.group(1).replace(" ", "").replace("\u00A0", "") if m_surface else None

    # surface jardin/terrain (span spécifique)
    ep_dtl_garden = ep_dtl.find("span", {"class": "dtl-main-surface-terrain"})
    if ep_dtl_garden:
        m_garden = RE_GARDEN.search(ep_dtl_garden.get_text(" ", strip=True))
        surface_garden = m_garden.group(0).replace(" ", "").replace("\u00A0", "") if m_garden else None
    else:
        surface_garden = None

    # nombre de pièces
    if ep_room:
        m_room = RE_ROOM.search(ep_room.get_text(" ", strip=True))
        room = m_room.group(0) if m_room else None
    else:
        room = None

    # ville + cp
    m_loc = RE_LOC.search(ep_loc.get_text(" ", strip=True))
    if not m_loc:
        return None

    ville = m_loc.group(1)
    code_postal = m_loc.group(2)

    # mapping surfaces selon type
    if type_bien == "terrain":
        surface_terrain = surface
        surface_interieure = None
    else:
        surface_interieure = surface
        surface_terrain = None

    return {
        "prix": price,
        "type_de_bien": type_bien,
        "url_annonce": href,
        "surface_terrain": surface_terrain,
        "surface_interieure": surface_interieure,
        "surface_jardin": surface_garden,
        "nombre_de_pieces": room,
        "ville": ville,
        "code_postal": code_postal,
    }

In [50]:

def collect_urls(
    lst_dep: list[str],
    list_prix_min: list[str],
    list_prix_max: list[str],
    bien_code: str,
    max_workers: int = 10,
) -> list[str]:
    """
    Collecte toutes les URLs d'annonces pour un type de bien,
    en parallélisant par département et tranche de prix.
    """

    price_pairs = list(zip(list_prix_min, list_prix_max))
    href_list: list[str] = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {}

        for dep in lst_dep:
            for prix_min, prix_max in price_pairs:
                fut = ex.submit(scrape_biens, dep, bien_code, prix_min, prix_max)
                futures[fut] = (dep, prix_min, prix_max)

        for fut in as_completed(futures):
            dep, prix_min, prix_max = futures[fut]
            try:
                hrefs = fut.result()
                href_list.extend(hrefs)
            except Exception as e:
                print(f"[dep={dep} prix={prix_min}{prix_max}] erreur: {e}")

    # dédoublonnage en conservant l’ordre
    href_list = list(dict.fromkeys(href_list))
    return href_list

In [ ]:
def parse_ads_parallel(
    href_list: list[str],
    extract_fn: Callable[[str, str], dict | None],
    info_bien_dic: dict[str, list],
    max_workers: int = 15,
    verbose: bool = False,
) -> tuple[list[dict], dict[str, list]]:
    """
    Parallélise l'extraction d'infos sur chaque URL d'annonce.

    - href_list : liste d'URLs
    - extract_fn : fonction du style extract_fn(href, var) -> dict | None
    - var : ex "terrain"
    - info_bien_dic : dict de listes à remplir
    - max_workers : nombre de threads
    - verbose : print chaque row si True

    Retourne : (results, info_bien_dic)
    """
    results: list[dict] = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(extract_fn, href): href for href in href_list}

        for fut in as_completed(futures):
            href = futures[fut]
            try:
                row = fut.result()
                if row is None:
                    continue
                if verbose:
                    print(row)
                results.append(row)
            except Exception as e:
                print(f"Problème avec l'annonce : {href} -> {e}")

    # Remplir le dict final (single-thread => pas de corruption)
    for row in results:
        for k in info_bien_dic:
            info_bien_dic[k].append(row.get(k))

    return info_bien_dic


In [52]:
bien_code = list_bien[2]
MAX_WORKERS = 10
href_list = collect_urls(
    lst_dep=lst_dep,
    list_prix_min=list_prix_min,
    list_prix_max=list_prix_max,
    bien_code=bien_code,
    max_workers=MAX_WORKERS,
)
print("Total hrefs:", len(href_list))

772
On cherche les annonces par date décroissante sur 9 pages (172 annonces)
783
On cherche les annonces par date décroissante sur 10 pages (183 annonces)
627
On cherche les annonces par date décroissante sur 2 pages (27 annonces)
632
On cherche les annonces par date décroissante sur 2 pages (32 annonces)
826
On cherche les annonces par date décroissante sur 12 pages (226 annonces)
851
On cherche les annonces par date décroissante sur 13 pages (251 annonces)
Total hrefs: 65447


In [53]:
href_list

['https://www.etreproprio.com/immobilier-23781119-vente-terrain-443m-a-sergy-sergy',
 'https://www.etreproprio.com/immobilier-23732669-vente-terrain-433m-a-sergy-sergy',
 'https://www.etreproprio.com/immobilier-23732667-vente-terrain-638m-a-pougny-pougny',
 'https://www.etreproprio.com/immobilier-23486274-vente-terrain-viabilise-724m2-sur-ae-peron-logras-peron',
 'https://www.etreproprio.com/immobilier-23465375-vente-grand-terrain-plat-libre-constructeur-au-calme-a-corbonod-seyssel',
 'https://www.etreproprio.com/immobilier-20318044-vente-terrain-constructible-libre-constructeur-pougny',
 'https://www.etreproprio.com/immobilier-22677726-vente-terrain-a-batir-762-m2-massieux',
 'https://www.etreproprio.com/immobilier-22628352-vente-terrain-591m-a-pougny-pougny',
 'https://www.etreproprio.com/immobilier-22601449-vente-terrain-constructible-d-environ-1-000m2-avec-2-gites-amberieu-en-bugey',
 'https://www.etreproprio.com/immobilier-22373996-vente-terrain-453m-a-thoiry-thoiry',
 'https://ww

In [54]:
MAX_WORKERS = 15
info_bien_dic = {
    "prix": [],
    "type_de_bien": [],
    "url_annonce": [],
    "surface_terrain": [],
    "surface_interieure": [],
    "surface_jardin": [],
    "nombre_de_pieces": [],
    "ville": [],
    "code_postal": [],
}
info_bien_dic = parse_ads_parallel(
    href_list=href_list,
    extract_fn=extract_fn,
    info_bien_dic = info_bien_dic,
    max_workers=MAX_WORKERS,
    verbose=False
)

Problème avec l'annonce : https://www.etreproprio.com/immobilier-23041192-vente-terrain-1660m-a-chaponost-chaponost -> ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Problème avec l'annonce : https://www.etreproprio.com/immobilier-22379873-vente-terrain-514m-a-rillieux-la-pape-rillieux-la-pape -> ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))


In [55]:
import csv

def dict_to_csv(data: dict[str, list], filename: str):
    keys = data.keys()
    rows = zip(*data.values())

    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(keys)   # header
        writer.writerows(rows)  # data

dict_to_csv(info_bien_dic, f"annonces{bien_code}.csv")


In [56]:
for items in info_bien_dic.items():
    print(f"{items[0]}: {len(items[1])}")

prix: 62150
type_de_bien: 62150
url_annonce: 62150
surface_terrain: 62150
surface_interieure: 62150
surface_jardin: 62150
nombre_de_pieces: 62150
ville: 62150
code_postal: 62150
